# Create a MatrixTable and QC the hell out of it
## Import stuff and set your parameters
First, we import necessary libraries and configurations from config.toml. Then we initialise Spark and Hail. 

In [10]:
!pip install toml
!pip install jupytext

In [1]:
# Imports
import toml
from pathlib import Path
from datetime import datetime
from pprint import pprint
from functools import partial
import sys
from distutils.version import LooseVersion
import subprocess

import pandas as pd
import pyspark
import dxpy
import dxdata
import hail as hl
    
module_path = Path('..').resolve().__str__()

if module_path not in sys.path:
    sys.path.append(module_path)

try:
    Path("../tmp").resolve().mkdir(parents = True, exists_ok = True)
except:
    pass
    
from src.utils import show_stats
from src.matrixtables import *

hl.plot.output_notebook()

Loading BokehJS ...

In [2]:
# Parameters
with open("../config.toml") as f:
    conf = toml.load(f)

IMPORT = conf["IMPORT"]
NAME = conf["NAME"]
VCF_VERSION = IMPORT["VCF_VERSION"]
REFERENCE_GENOME = conf["REFERENCE_GENOME"]
DATABASE = IMPORT["DATABASE"]

LOG_FILE = Path(IMPORT["LOG_DIR"],f"{NAME}_{datetime.now().strftime('%H%M')}.log").resolve().__str__()
MAP_FILE = Path(IMPORT["MAPPING_FILE"]).resolve().__str__()
INT_FILE = Path(IMPORT["INTERVAL_FILE"]).resolve().__str__()
GENE_FILE = Path(IMPORT["GENE_FILE"]).resolve().__str__()
FILTER_FILE = Path(conf["SAMPLE_QC"]["DATA_DIR"], conf["SAMPLE_QC"]["SAMPLE_FILTER_FILE"]).resolve().__str__()

VCF_DIR = Path(IMPORT["VCF_DIR"]).resolve().__str__()

DOWNSAMPLE_P = IMPORT.get("DOWNSAMPLE_P", None)

SNV_ONLY = conf["ANNOTATE"]["SNV_ONLY"]
USE_VEP = conf["ANNOTATE"]["USE_VEP"]
MISSENSE_ONLY = conf["ANNOTATE"]["MISSENSE_ONLY"]

VEP_JSON = Path(conf["ANNOTATE"]["VEP_JSON"]).resolve().__str__()

ANNOTATION_DIR = conf["ANNOTATE"]["ANNOTATION_DIR"]

MIN_DP = conf["ENTRY_QC"]["MIN_DP"]
MIN_GQ = conf["ENTRY_QC"]["MIN_GQ"]
MIN_PL = conf["ENTRY_QC"]["MIN_PL"]

MIN_P_HWE = conf["VARIANT_QC"]["MIN_P_HWE"]
MIN_VAR_GQ = conf["VARIANT_QC"]["MIN_VAR_GQ"]

MIN_CALL_RATE = conf["SAMPLE_QC"]["MIN_CALL_RATE"]
MIN_MEAN_DP = conf["SAMPLE_QC"]["MIN_MEAN_DP"]
MIN_MEAN_GQ = conf["SAMPLE_QC"]["MIN_MEAN_GQ"]

TMP_DIR = conf["EXPORT"]["TMP_DIR"]

BGEN_FILE = Path(TMP_DIR, f"{NAME}").resolve().__str__()
ANNOTATIONS_FILE = Path(TMP_DIR, f"{NAME}.annotations").resolve().__str__()
SETLIST_FILE = Path(TMP_DIR, f"{NAME}.setlist").resolve().__str__()


In [3]:
# Spark and Hail

sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

try:
    mt_database = dxpy.find_one_data_object(name=DATABASE)["id"]
except Exception as e:
    print(e.message)
    spark.sql(f"CREATE DATABASE {DATABASE} LOCATION  'dnax://'")
    mt_database = dxpy.find_one_data_object(name=DATABASE)["id"]
    
# this breaks export_bgen for now
# hl.init(sc=sc, default_reference=REFERENCE_GENOME, log=LOG_FILE, tmp_dir=f'dnax://{mt_database}/tmp/')

hl.init(sc=sc, default_reference=REFERENCE_GENOME, log=LOG_FILE)

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.4
SparkUI available at http://ip-10-60-0-47.eu-west-2.compute.internal:8081
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /opt/notebooks/gogoGPCR/hail_logs/DRD2_1334.log


In [4]:
with open(GENE_FILE, "r") as file:
    genes = file.read().splitlines()
    
mapping = pd.read_csv(MAP_FILE, sep = "\t").set_index("HGNC", drop=False)

In [5]:
# Import
mt = import_mt(genes, mapping, vcf_dir = VCF_DIR, vcf_version = VCF_VERSION).key_rows_by("locus", "alleles")#.checkpoint(checkpoint_file)

v, s = mt.count()
pprint(f"{v} variants and {s} samples after import")

'582 variants and 200643 samples after import'


In [6]:
# Checkpoint
stage = "raw"
checkpoint_file = f"/tmp/{NAME}.{stage}.cp.mt"

mt = mt.checkpoint(checkpoint_file, overwrite = True)

2021-11-08 13:38:39 Hail: INFO: Coerced sorted dataset
2021-11-08 13:40:44 Hail: INFO: wrote matrix table with 582 rows and 200643 columns in 1 partition to /tmp/DRD2.raw.cp.mt
    Total size: 602.72 MiB
    * Rows/entries: 601.41 MiB
    * Columns: 1.31 MiB
    * Globals: 11.00 B
    * Smallest partition: 582 rows (601.41 MiB)
    * Largest partition:  582 rows (601.41 MiB)


In [7]:
# Downsample
if DOWNSAMPLE_P is not None:
    mt = downsample_mt(mt, DOWNSAMPLE_P)

    pprint(f"{mt.count_cols()} samples after downsampling")

In [8]:
def interval_qc_mt(
    mt: hl.matrixtable.MatrixTable,
    bed_file: Union[str, Path],
) -> hl.matrixtable.MatrixTable:
    """Filter to only Target region used by the WES capture experiment

    Parameters
    ----------
    mt : hl.matrixtable.MatrixTable
        MatrixTable
    intervals : str
        .BED file of targeted capture regions which meet quality standards

    Returns
    -------
    hl.matrixtable.MatrixTable
        MatrixTable filtered to only target regions
    """

    interval_table = hl.import_bed(
        bed_file,
        reference_genome="GRCh38",
        #filter=f"^(?!(chr{mapping['GRCh38_region']}))",
    )

    mt = mt.filter_rows(hl.is_defined(interval_table[mt.locus]))

    return mt

In [9]:
# Interval QC
mt = interval_qc_mt(mt, "file:" + INT_FILE)

pprint(f"{mt.count_rows()} variants after interval filtering")

2021-11-08 13:40:45 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (user-supplied)
  Loading field 'f1' as type int32 (user-supplied)
  Loading field 'f2' as type int32 (user-supplied)
2021-11-08 13:40:47 Hail: INFO: Coerced sorted dataset


'276 variants after interval filtering'


In [10]:
# Split multi
mt = mt.filter_rows(mt.alleles.length() <= 6)
mt = smart_split_multi_mt(mt)

pprint(f'{mt.count_rows()} variants with not more than 6 alleles after splitting')

2021-11-08 13:40:51 Hail: INFO: Coerced sorted dataset
2021-11-08 13:40:54 Hail: INFO: Coerced sorted dataset
2021-11-08 13:40:57 Hail: INFO: Coerced sorted dataset


'297 variants with not more than 6 alleles after splitting'


In [11]:
if USE_VEP:
    mt = hl.vep(mt, "file:" + VEP_JSON)
    
    is_MANE = mt.aggregate_rows(hl.agg.all(hl.is_defined(mt.vep.transcript_consequences.mane_select)))
    assert is_MANE, "Selected transcript may not be MANE Select. Check manually."
    
    mt = mt.annotate_rows(protCons = mt.vep.transcript_consequences.amino_acids[0].split("/")[0] +
                       hl.str(mt.vep.transcript_consequences.protein_end[0]) +
                       mt.vep.transcript_consequences.amino_acids[0].split("/")[-1])
    

2021-10-29 12:43:57 Hail: INFO: Coerced sorted dataset
2021-10-29 12:43:58 Hail: INFO: Coerced sorted dataset
2021-10-29 12:44:00 Hail: INFO: Coerced sorted dataset
2021-10-29 12:44:01 Hail: INFO: Coerced sorted dataset
2021-10-29 12:44:07 Hail: INFO: Coerced sorted dataset
2021-10-29 12:44:08 Hail: INFO: Coerced sorted dataset


In [21]:
ANNOTATION_FILE = Path(ANNOTATION_DIR, f"{NAME}.tsv").resolve().__str__()
mt = annotate_mt(mt = mt, gene = NAME, annotations = ANNOTATION_FILE)

interesting = mt.filter_rows((hl.is_defined(mt.annotations)) & (hl.agg.any(mt.GT.is_non_ref()))).count_rows()
pprint(f"{interesting} annotated variants found before QC")

2021-11-08 13:47:50 Hail: WARN: '/mnt/project/annotations/DRD2.tsv' refers to no files


FatalError: HailException: arguments refer to no files

Java stack trace:
is.hail.utils.HailException: arguments refer to no files
	at is.hail.utils.ErrorHandling$class.fatal(ErrorHandling.scala:11)
	at is.hail.utils.package$.fatal(package.scala:77)
	at is.hail.expr.ir.TextTableReader$.readMetadata(TextTableReader.scala:253)
	at is.hail.expr.ir.TextTableReader$.apply(TextTableReader.scala:306)
	at is.hail.expr.ir.TextTableReader$.fromJValue(TextTableReader.scala:313)
	at is.hail.expr.ir.TableReader$.fromJValue(TableIR.scala:103)
	at is.hail.expr.ir.IRParser$.table_ir_1(Parser.scala:1462)
	at is.hail.expr.ir.IRParser$$anonfun$table_ir$1.apply(Parser.scala:1438)
	at is.hail.expr.ir.IRParser$$anonfun$table_ir$1.apply(Parser.scala:1438)
	at is.hail.utils.StackSafe$More.advance(StackSafe.scala:64)
	at is.hail.utils.StackSafe$.run(StackSafe.scala:16)
	at is.hail.utils.StackSafe$StackFrame.run(StackSafe.scala:32)
	at is.hail.expr.ir.IRParser$$anonfun$parse_table_ir$1.apply(Parser.scala:1957)


In [12]:
# Checkpoint
stage = "QC1"
checkpoint_file = f"/tmp/{NAME}.{stage}.cp.mt"

mt = mt.checkpoint(checkpoint_file, overwrite = True)
#show_stats(mt)

FatalError: HailException: path '/tmp/DRD2.QC1.cp.mt' is both an input and output source in a single query. Write, export, or checkpoint to a different path!

Java stack trace:
is.hail.utils.HailException: path '/tmp/DRD2.QC1.cp.mt' is both an input and output source in a single query. Write, export, or checkpoint to a different path!
	at is.hail.utils.ErrorHandling$class.fatal(ErrorHandling.scala:11)
	at is.hail.utils.package$.fatal(package.scala:77)
	at is.hail.expr.Validate$.fileReadWriteError(Validate.scala:13)
	at is.hail.expr.Validate$$anonfun$is$hail$expr$Validate$$validate$2.apply(Validate.scala:24)
	at is.hail.expr.Validate$$anonfun$is$hail$expr$Validate$$validate$2.apply(Validate.scala:22)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.WrappedArray.foreach(WrappedArray.scala:35)
	at is.hail.expr.Validate$.is$hail$expr$Validate$$validate(Validate.scala:22)
	at is.hail.expr.Validate$.apply(Validate.scala:10)
	at

In [13]:
# Withdrawn
mt = mt.filter_cols(~mt.s.startswith("W"))

print(f"Samples remaining after removing withdrawn participants: {mt.count_cols()} ")

Samples remaining after removing withdrawn participants: 200611 


In [17]:
# Filter samples
samples_to_remove =  hl.import_table("file:" + FILTER_FILE, key = "eid")
mt = mt.anti_join_cols(samples_to_remove)
print(f"Samples remaining after removing related samples: {mt.count_cols()} ")

2021-11-08 13:44:23 Hail: INFO: Reading table without type imputation
  Loading field 'eid' as type str (not specified)


Samples remaining after removing related samples: 154576 


In [18]:
# Sample QC
mt = sample_QC_mt(mt, MIN_CALL_RATE, MIN_MEAN_DP, MIN_MEAN_GQ)

print(f"Samples remaining after QC: {mt.count_cols()} ")

Samples remaining after QC: 150636 


In [21]:
# Variant QC
mt = variant_QC_mt(mt, MIN_P_HWE, MIN_VAR_GQ)

interesting = mt.filter_rows((hl.is_defined(mt.annotations)) & (hl.agg.any(mt.GT.is_non_ref()))).count_rows()
print(f"{mt.count_rows()} variants remaining after QC of which {interesting} are annotated")

211 variants remaining after QC of which 76 are annotated


In [22]:
# Genotype GQ
mt = genotype_filter_mt(mt, MIN_DP, MIN_GQ, True)

missing = mt.aggregate_entries(hl.agg.sum(~hl.is_defined(mt.GT)))
pprint(f"{missing} missing or filtered entries after Call QC")

Filtering 0.00% entries out of downstream analysis.
'0 missing or filtered entries after Call QC'


In [23]:
# Checkpoint
stage = "QC2"
checkpoint_file = f"/tmp/{GENE}.{stage}.cp.mt"

mt = mt.checkpoint(checkpoint_file, overwrite = True)
show_stats(mt)

2021-10-29 12:50:05 Hail: INFO: wrote matrix table with 211 rows and 150347 columns in 2 partitions to /tmp/DRD2.QC2.cp.mt
    Total size: 171.36 MiB
    * Rows/entries: 164.09 MiB
    * Columns: 7.28 MiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  211 rows (164.09 MiB)


'Stats before QC:'


2021-10-29 12:50:09 Hail: INFO: Ordering unsorted dataset with network shuffle


,,
annotation,n_carriers,n_variants
str,int64,int64
"""Gi1""",110,14
"""Gi1_GoA_Gz""",2,1
"""Gi1_Gz""",2,2
"""Gz""",5,1
"""WT""",7496,58


In [24]:
# BGEN
write_bgen(mt, "file:" + BGEN_FILE)

2021-10-29 12:50:24 Hail: INFO: while writing:
    file:/opt/notebooks/gogoGPCR/tmp/DRD2.bgen
  merge time: 41.414ms


In [25]:
# ANNOTATIONS

mt = add_varid(mt)

annotations = (
    mt.select_rows(
        varid = mt.varid,
        gene = mt.vep.transcript_consequences.gene_symbol[0],
        annotation = mt.annotation
    )
    .rows()
    .key_by("varid")
    .drop("locus")
    .drop("alleles")
)
annotations.export("file:" + ANNOTATIONS_FILE, header=False)

2021-10-29 12:50:24 Hail: INFO: Coerced sorted dataset
2021-10-29 12:50:25 Hail: INFO: merging 1 files totalling 5.8K...
2021-10-29 12:50:25 Hail: INFO: while writing:
    file:/opt/notebooks/gogoGPCR/tmp/DRD2.annotations
  merge time: 14.078ms


In [26]:
# SETLIST
position = mt.aggregate_rows(hl.agg.min(mt.locus.position))
names = mt.varid.collect()
names_str = ",".join(names)

line = f"{mt.vep.transcript_consequences.gene_symbol[0].collect()[0]}\t{mt.locus.contig.collect()[0]}\t{position}\t{names_str}"

with open(SETLIST_FILE, "w") as f:
    f.write(line)

In [27]:
bgen_file = BGEN_FILE + ".bgen"
sample_file = BGEN_FILE + ".sample"

#subprocess.run(["dx", "upload", bgen_file, sample_file, ANNOTATIONS_FILE, SETLIST_FILE, "--path", "/data/burden/"], check = True, shell = False)

In [28]:
sample = mt.select_cols(ID_1 = mt.s, ID_2 = mt.s, missing = 0)

In [29]:
sample.cols().show()

2021-10-29 12:50:26 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'
2021-10-29 12:50:26 Hail: INFO: Coerced sorted dataset


,,,
s,ID_1,ID_2,missing
str,str,str,int32
"""1000030""","""1000030""","""1000030""",0
"""1000059""","""1000059""","""1000059""",0
"""1000062""","""1000062""","""1000062""",0
"""1000077""","""1000077""","""1000077""",0
"""1000086""","""1000086""","""1000086""",0
"""1000100""","""1000100""","""1000100""",0
"""1000229""","""1000229""","""1000229""",0
"""1000250""","""1000250""","""1000250""",0


In [30]:
#STAGE = "final"
#WRITE_PATH = "dnax://" + mt_database + f"/{GENE}.{STAGE}.mt"

#mt.write(WRITE_PATH, overwrite = True)
show_stats(mt)

#STAGE = "final"
#WRITE_PATH = "dnax://" + mt_database + f"/{GENE}.{STAGE}.mt"

#mt = hl.read_matrix_table(WRITE_PATH)

'Stats before QC:'


2021-10-29 12:50:30 Hail: INFO: Ordering unsorted dataset with network shuffle


,,
annotation,n_carriers,n_variants
str,int64,int64
"""Gi1""",110,14
"""Gi1_GoA_Gz""",2,1
"""Gi1_Gz""",2,2
"""Gz""",5,1
"""WT""",7496,58
